# 02 - EDA: Battle Metadata

**Purpose**: Explore battle-level patterns across arenas, game modes, and trophy levels.

**Key Questions**:
- How are battles distributed across arenas?
- Do win rates vary by trophy level?
- Are certain game modes more popular?
- What's the average trophy change per battle?
- How does crown count correlate with trophy gains?

**Potential Insights for Presentation**:
- Trophy progression walls (where players get stuck)
- Game mode preferences by skill level
- Risk/reward analysis (trophy gains vs. matchup difficulty)

In [1]:
# Setup
import sys, os
import duckdb, pandas as pd, numpy as np
import matplotlib.pyplot as plt, seaborn as sns

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, os.path.join(PROJECT_ROOT, 'src'))

# Use Parquet if available (faster), fallback to CSV
DATA_PATH = os.path.join(PROJECT_ROOT, 'battles.parquet')
if not os.path.exists(DATA_PATH):
    DATA_PATH = os.path.join(PROJECT_ROOT, 'battles.csv')

from duckdb_utils import get_connection, create_battles_view, query_to_df
from visualization import setup_presentation_style

con = get_connection()
create_battles_view(con, DATA_PATH)
setup_presentation_style()

print("✓ Ready for analysis")

✓ Created view 'battles' from Parquet: c:\Users\Danny\Documents\GitHub\HeHeHaHa_DataRoyale\battles.parquet
✓ Presentation style configured
✓ Ready for analysis


## 1. Arena Analysis

In [ ]:
# Analyze battle distribution by arena
arena_stats = query_to_df(con, """
    SELECT 
        "arena.id" as arena_id,
        COUNT(*) as battles,
        ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (), 2) as pct_of_total,
        AVG("winner.trophyChange") as avg_trophy_gain,
        AVG("winner.crowns") as avg_crowns,
        AVG("average.startingTrophies") as avg_trophy_level,
        MIN("average.startingTrophies") as min_trophies,
        MAX("average.startingTrophies") as max_trophies
    FROM battles
    WHERE "arena.id" IS NOT NULL
    GROUP BY "arena.id"
    ORDER BY battles DESC
""")

print("Arena Battle Distribution:")
print(f"Total arenas: {len(arena_stats)}")
print(f"\nTop 10 Arenas by Battle Count:")
print(arena_stats.head(10).to_string(index=False))

# Visualize arena distribution
plt.figure(figsize=(14, 8))

# Plot 1: Battle count by arena
plt.subplot(2, 2, 1)
top_arenas = arena_stats.head(15)
plt.barh(range(len(top_arenas)), top_arenas['battles'], color='steelblue', alpha=0.7)
plt.yticks(range(len(top_arenas)), top_arenas['arena_id'])
plt.xlabel('Number of Battles', fontsize=12)
plt.ylabel('Arena ID', fontsize=12)
plt.title('Battle Distribution by Arena (Top 15)', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.grid(axis='x', alpha=0.3)

# Plot 2: Average trophy gain by arena
plt.subplot(2, 2, 2)
plt.scatter(arena_stats['battles'], arena_stats['avg_trophy_gain'], 
           s=100, alpha=0.6, color='coral', edgecolors='black', linewidth=0.5)
plt.xlabel('Number of Battles (log scale)', fontsize=12)
plt.ylabel('Average Trophy Gain', fontsize=12)
plt.title('Trophy Gain vs Battle Volume', fontsize=14, fontweight='bold')
plt.xscale('log')
plt.grid(alpha=0.3)

# Plot 3: Average crowns by arena
plt.subplot(2, 2, 3)
plt.bar(range(len(top_arenas)), top_arenas['avg_crowns'], color='darkgreen', alpha=0.7)
plt.xticks(range(len(top_arenas)), top_arenas['arena_id'], rotation=45, ha='right')
plt.xlabel('Arena ID', fontsize=12)
plt.ylabel('Average Crowns', fontsize=12)
plt.title('Average Crowns per Win by Arena', fontsize=14, fontweight='bold')
plt.grid(axis='y', alpha=0.3)

# Plot 4: Trophy level range by arena
plt.subplot(2, 2, 4)
for idx, row in top_arenas.iterrows():
    plt.plot([row['min_trophies'], row['max_trophies']], 
             [idx, idx], 'o-', linewidth=2, markersize=8, alpha=0.7)
plt.xlabel('Trophy Range', fontsize=12)
plt.ylabel('Arena Index', fontsize=12)
plt.title('Trophy Level Range by Arena', fontsize=14, fontweight='bold')
plt.yticks(range(len(top_arenas)), top_arenas['arena_id'])
plt.grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Key insights
dominant_arena = arena_stats.iloc[0]
print(f"\n📊 Key Findings:")
print(f"  • Dominant arena: {dominant_arena['arena_id']} ({dominant_arena['pct_of_total']:.1f}% of all battles)")
print(f"  • Average trophy gain across all arenas: {arena_stats['avg_trophy_gain'].mean():.2f}")
print(f"  • Average crowns per win: {arena_stats['avg_crowns'].mean():.2f}")

## 2. Trophy Level Patterns

In [ ]:
# Create trophy bracket analysis
# Look for "walls" where players cluster (trophy progression bottlenecks)
trophy_brackets = query_to_df(con, """
    WITH brackets AS (
        SELECT 
            CASE 
                WHEN "average.startingTrophies" < 1000 THEN '0-1000'
                WHEN "average.startingTrophies" < 2000 THEN '1000-2000'
                WHEN "average.startingTrophies" < 3000 THEN '2000-3000'
                WHEN "average.startingTrophies" < 4000 THEN '3000-4000'
                WHEN "average.startingTrophies" < 5000 THEN '4000-5000'
                WHEN "average.startingTrophies" < 6000 THEN '5000-6000'
                WHEN "average.startingTrophies" < 7000 THEN '6000-7000'
                ELSE '7000+'
            END as trophy_bracket,
            "average.startingTrophies" as avg_trophies,
            "winner.trophyChange" as trophy_change,
            "winner.crowns" as crowns
        FROM battles
        WHERE "average.startingTrophies" IS NOT NULL
    )
    SELECT 
        trophy_bracket,
        COUNT(*) as battle_count,
        ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (), 2) as pct_of_total,
        AVG(avg_trophies) as avg_trophy_level,
        AVG(trophy_change) as avg_trophy_gain,
        STDDEV(trophy_change) as std_trophy_gain,
        AVG(crowns) as avg_crowns,
        PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY trophy_change) as q1_trophy_gain,
        PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY trophy_change) as median_trophy_gain,
        PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY trophy_change) as q3_trophy_gain
    FROM brackets
    GROUP BY trophy_bracket
    ORDER BY MIN(CASE 
        WHEN trophy_bracket = '0-1000' THEN 1
        WHEN trophy_bracket = '1000-2000' THEN 2
        WHEN trophy_bracket = '2000-3000' THEN 3
        WHEN trophy_bracket = '3000-4000' THEN 4
        WHEN trophy_bracket = '4000-5000' THEN 5
        WHEN trophy_bracket = '5000-6000' THEN 6
        WHEN trophy_bracket = '6000-7000' THEN 7
        ELSE 8
    END)
""")

print("Trophy Bracket Analysis:")
print(trophy_brackets.to_string(index=False))

# Visualize trophy brackets
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Plot 1: Battle distribution by bracket
axes[0, 0].bar(range(len(trophy_brackets)), trophy_brackets['pct_of_total'], 
               color='steelblue', alpha=0.7, edgecolor='black')
axes[0, 0].set_xticks(range(len(trophy_brackets)))
axes[0, 0].set_xticklabels(trophy_brackets['trophy_bracket'], rotation=45, ha='right')
axes[0, 0].set_ylabel('Percentage of Battles (%)', fontsize=12)
axes[0, 0].set_title('Battle Distribution by Trophy Bracket', fontsize=14, fontweight='bold')
axes[0, 0].grid(axis='y', alpha=0.3)

# Add value labels
for i, pct in enumerate(trophy_brackets['pct_of_total']):
    axes[0, 0].text(i, pct, f'{pct:.1f}%', ha='center', va='bottom', fontsize=10)

# Plot 2: Average trophy gain by bracket
axes[0, 1].plot(range(len(trophy_brackets)), trophy_brackets['avg_trophy_gain'], 
                marker='o', linewidth=2, markersize=10, color='darkgreen')
axes[0, 1].fill_between(range(len(trophy_brackets)), 
                        trophy_brackets['avg_trophy_gain'] - trophy_brackets['std_trophy_gain'],
                        trophy_brackets['avg_trophy_gain'] + trophy_brackets['std_trophy_gain'],
                        alpha=0.3, color='green')
axes[0, 1].axhline(y=0, color='red', linestyle='--', linewidth=1, alpha=0.5)
axes[0, 1].set_xticks(range(len(trophy_brackets)))
axes[0, 1].set_xticklabels(trophy_brackets['trophy_bracket'], rotation=45, ha='right')
axes[0, 1].set_ylabel('Average Trophy Gain', fontsize=12)
axes[0, 1].set_title('Average Trophy Gain by Bracket (±1 Std Dev)', fontsize=14, fontweight='bold')
axes[0, 1].grid(alpha=0.3)

# Plot 3: Battle count (absolute)
axes[1, 0].bar(range(len(trophy_brackets)), trophy_brackets['battle_count'], 
               color='coral', alpha=0.7, edgecolor='black')
axes[1, 0].set_xticks(range(len(trophy_brackets)))
axes[1, 0].set_xticklabels(trophy_brackets['trophy_bracket'], rotation=45, ha='right')
axes[1, 0].set_ylabel('Number of Battles', fontsize=12)
axes[1, 0].set_title('Battle Count by Trophy Bracket', fontsize=14, fontweight='bold')
axes[1, 0].grid(axis='y', alpha=0.3)
axes[1, 0].set_yscale('log')

# Plot 4: Average crowns by bracket
axes[1, 1].bar(range(len(trophy_brackets)), trophy_brackets['avg_crowns'], 
               color='purple', alpha=0.7, edgecolor='black')
axes[1, 1].set_xticks(range(len(trophy_brackets)))
axes[1, 1].set_xticklabels(trophy_brackets['trophy_bracket'], rotation=45, ha='right')
axes[1, 1].set_ylabel('Average Crowns per Win', fontsize=12)
axes[1, 1].set_title('Average Crowns by Trophy Bracket', fontsize=14, fontweight='bold')
axes[1, 1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

# Identify trophy walls (brackets with high battle concentration)
print(f"\n📊 Trophy Progression Analysis:")
max_bracket = trophy_brackets.loc[trophy_brackets['battle_count'].idxmax()]
print(f"  • Most battles in bracket: {max_bracket['trophy_bracket']} ({max_bracket['pct_of_total']:.1f}% of all battles)")
print(f"  • This suggests a potential 'trophy wall' where players cluster")

# Check for progression bottlenecks (low trophy gain relative to battle volume)
trophy_brackets['efficiency_ratio'] = trophy_brackets['avg_trophy_gain'] / (trophy_brackets['battle_count'] / 1000000)
bottleneck = trophy_brackets.loc[trophy_brackets['efficiency_ratio'].idxmin()]
print(f"  • Least efficient bracket: {bottleneck['trophy_bracket']} (low trophy gain relative to battle volume)")
print(f"  • Average trophy gain: {bottleneck['avg_trophy_gain']:.2f}")

# Show correlation between trophy level and gain
correlation = trophy_brackets['avg_trophy_level'].corr(trophy_brackets['avg_trophy_gain'])
print(f"  • Correlation between trophy level and gain: {correlation:.3f}")

## 3. Game Mode Insights

In [ ]:
# Compare game modes
game_mode_stats = query_to_df(con, """
    SELECT 
        "gameMode.id" as game_mode,
        COUNT(*) as battle_count,
        ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (), 2) as pct_of_total,
        AVG("winner.trophyChange") as avg_trophy_gain,
        STDDEV("winner.trophyChange") as std_trophy_gain,
        AVG("winner.crowns") as avg_crowns,
        AVG("average.startingTrophies") as avg_trophy_level,
        PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY "winner.trophyChange") as q1_trophy_gain,
        PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY "winner.trophyChange") as median_trophy_gain,
        PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY "winner.trophyChange") as q3_trophy_gain
    FROM battles
    WHERE "gameMode.id" IS NOT NULL
    GROUP BY "gameMode.id"
    ORDER BY battle_count DESC
""")

print("Game Mode Analysis:")
print(game_mode_stats.to_string(index=False))

# Visualize game mode comparison
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Plot 1: Battle distribution by game mode
axes[0, 0].bar(range(len(game_mode_stats)), game_mode_stats['pct_of_total'], 
               color='steelblue', alpha=0.7, edgecolor='black')
axes[0, 0].set_xticks(range(len(game_mode_stats)))
axes[0, 0].set_xticklabels(game_mode_stats['game_mode'], rotation=45, ha='right')
axes[0, 0].set_ylabel('Percentage of Battles (%)', fontsize=12)
axes[0, 0].set_title('Battle Distribution by Game Mode', fontsize=14, fontweight='bold')
axes[0, 0].grid(axis='y', alpha=0.3)

# Add value labels
for i, pct in enumerate(game_mode_stats['pct_of_total']):
    axes[0, 0].text(i, pct, f'{pct:.1f}%', ha='center', va='bottom', fontsize=11)

# Plot 2: Average trophy gain by game mode (with error bars)
axes[0, 1].bar(range(len(game_mode_stats)), game_mode_stats['avg_trophy_gain'], 
               yerr=game_mode_stats['std_trophy_gain'],
               color='darkgreen', alpha=0.7, edgecolor='black', capsize=5)
axes[0, 1].axhline(y=0, color='red', linestyle='--', linewidth=1, alpha=0.5)
axes[0, 1].set_xticks(range(len(game_mode_stats)))
axes[0, 1].set_xticklabels(game_mode_stats['game_mode'], rotation=45, ha='right')
axes[0, 1].set_ylabel('Average Trophy Gain', fontsize=12)
axes[0, 1].set_title('Average Trophy Gain by Game Mode (±1 Std Dev)', fontsize=14, fontweight='bold')
axes[0, 1].grid(axis='y', alpha=0.3)

# Plot 3: Box plot data preparation (trophy gain distribution)
trophy_gain_by_mode = query_to_df(con, """
    SELECT 
        "gameMode.id" as game_mode,
        "winner.trophyChange" as trophy_gain
    FROM battles
    WHERE "gameMode.id" IS NOT NULL 
      AND "winner.trophyChange" IS NOT NULL
    LIMIT 50000  -- Sample for visualization
""")

# Create box plot
mode_list = game_mode_stats['game_mode'].tolist()
box_data = [trophy_gain_by_mode[trophy_gain_by_mode['game_mode'] == mode]['trophy_gain'].values 
            for mode in mode_list]
axes[1, 0].boxplot(box_data, labels=mode_list)
axes[1, 0].axhline(y=0, color='red', linestyle='--', linewidth=1, alpha=0.5)
axes[1, 0].set_ylabel('Trophy Gain', fontsize=12)
axes[1, 0].set_title('Trophy Gain Distribution by Game Mode', fontsize=14, fontweight='bold')
axes[1, 0].tick_params(axis='x', rotation=45)
axes[1, 0].grid(axis='y', alpha=0.3)

# Plot 4: Average crowns by game mode
axes[1, 1].bar(range(len(game_mode_stats)), game_mode_stats['avg_crowns'], 
               color='purple', alpha=0.7, edgecolor='black')
axes[1, 1].set_xticks(range(len(game_mode_stats)))
axes[1, 1].set_xticklabels(game_mode_stats['game_mode'], rotation=45, ha='right')
axes[1, 1].set_ylabel('Average Crowns per Win', fontsize=12)
axes[1, 1].set_title('Average Crowns by Game Mode', fontsize=14, fontweight='bold')
axes[1, 1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

# Risk/reward analysis
print(f"\n📊 Game Mode Risk/Reward Analysis:")
for _, row in game_mode_stats.iterrows():
    risk = row['std_trophy_gain']  # Higher std = higher risk
    reward = row['avg_trophy_gain']  # Average gain = reward
    print(f"  • {row['game_mode']}:")
    print(f"    - Risk (std dev): {risk:.2f}")
    print(f"    - Reward (avg gain): {reward:.2f}")
    print(f"    - Risk/Reward ratio: {risk/abs(reward) if reward != 0 else 'N/A':.2f}")

# Identify most popular and most rewarding modes
most_popular = game_mode_stats.iloc[0]
most_rewarding = game_mode_stats.loc[game_mode_stats['avg_trophy_gain'].idxmax()]
print(f"\n  • Most popular mode: {most_popular['game_mode']} ({most_popular['pct_of_total']:.1f}% of battles)")
print(f"  • Most rewarding mode: {most_rewarding['game_mode']} (avg gain: {most_rewarding['avg_trophy_gain']:.2f})")

## 4. Crown Analysis

In [ ]:
# Analyze crown patterns
crown_analysis = query_to_df(con, """
    SELECT 
        "winner.crowns" as crowns,
        COUNT(*) as battle_count,
        ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (), 2) as pct_of_total,
        AVG("winner.trophyChange") as avg_trophy_gain,
        STDDEV("winner.trophyChange") as std_trophy_gain,
        MIN("winner.trophyChange") as min_trophy_gain,
        MAX("winner.trophyChange") as max_trophy_gain,
        PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY "winner.trophyChange") as q1_trophy_gain,
        PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY "winner.trophyChange") as median_trophy_gain,
        PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY "winner.trophyChange") as q3_trophy_gain
    FROM battles
    WHERE "winner.crowns" IS NOT NULL
    GROUP BY "winner.crowns"
    ORDER BY "winner.crowns"
""")

print("Crown Win Analysis:")
print(crown_analysis.to_string(index=False))

# Visualize crown patterns
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Plot 1: Distribution of crown wins
axes[0, 0].bar(crown_analysis['crowns'], crown_analysis['pct_of_total'], 
               color=['#ff6b6b', '#4ecdc4', '#45b7d1'], alpha=0.7, edgecolor='black')
axes[0, 0].set_xlabel('Crowns', fontsize=12)
axes[0, 0].set_ylabel('Percentage of Battles (%)', fontsize=12)
axes[0, 0].set_title('Distribution of Crown Wins', fontsize=14, fontweight='bold')
axes[0, 0].set_xticks(crown_analysis['crowns'])
axes[0, 0].grid(axis='y', alpha=0.3)

# Add value labels
for i, (crown, pct) in enumerate(zip(crown_analysis['crowns'], crown_analysis['pct_of_total'])):
    axes[0, 0].text(crown, pct, f'{pct:.1f}%', ha='center', va='bottom', fontsize=12, fontweight='bold')

# Plot 2: Average trophy gain by crown count
axes[0, 1].plot(crown_analysis['crowns'], crown_analysis['avg_trophy_gain'], 
                marker='o', linewidth=3, markersize=12, color='darkgreen')
axes[0, 1].fill_between(crown_analysis['crowns'], 
                        crown_analysis['avg_trophy_gain'] - crown_analysis['std_trophy_gain'],
                        crown_analysis['avg_trophy_gain'] + crown_analysis['std_trophy_gain'],
                        alpha=0.3, color='green')
axes[0, 1].set_xlabel('Crowns', fontsize=12)
axes[0, 1].set_ylabel('Average Trophy Gain', fontsize=12)
axes[0, 1].set_title('Trophy Gain by Crown Count (±1 Std Dev)', fontsize=14, fontweight='bold')
axes[0, 1].set_xticks(crown_analysis['crowns'])
axes[0, 1].grid(alpha=0.3)

# Plot 3: Trophy gain distribution by crown count (box plot)
crown_trophy_data = query_to_df(con, """
    SELECT 
        "winner.crowns" as crowns,
        "winner.trophyChange" as trophy_gain
    FROM battles
    WHERE "winner.crowns" IS NOT NULL 
      AND "winner.trophyChange" IS NOT NULL
    LIMIT 100000  -- Sample for visualization
""")

crown_list = sorted(crown_trophy_data['crowns'].unique())
box_data = [crown_trophy_data[crown_trophy_data['crowns'] == c]['trophy_gain'].values 
            for c in crown_list]
axes[1, 0].boxplot(box_data, labels=crown_list)
axes[1, 0].axhline(y=0, color='red', linestyle='--', linewidth=1, alpha=0.5)
axes[1, 0].set_xlabel('Crowns', fontsize=12)
axes[1, 0].set_ylabel('Trophy Gain', fontsize=12)
axes[1, 0].set_title('Trophy Gain Distribution by Crown Count', fontsize=14, fontweight='bold')
axes[1, 0].grid(axis='y', alpha=0.3)

# Plot 4: Correlation scatter
for crown in crown_list:
    crown_data = crown_trophy_data[crown_trophy_data['crowns'] == crown]
    axes[1, 1].scatter([crown] * len(crown_data), crown_data['trophy_gain'], 
                       alpha=0.1, s=10, label=f'{crown} crown(s)')

axes[1, 1].set_xlabel('Crowns', fontsize=12)
axes[1, 1].set_ylabel('Trophy Gain', fontsize=12)
axes[1, 1].set_title('Crowns vs Trophy Gain (Scatter)', fontsize=14, fontweight='bold')
axes[1, 1].set_xticks(crown_list)
axes[1, 1].grid(alpha=0.3)
axes[1, 1].legend()

plt.tight_layout()
plt.show()

# Calculate correlation
correlation = crown_analysis['crowns'].corr(crown_analysis['avg_trophy_gain'])
print(f"\n📊 Crown Analysis Summary:")
print(f"  • Correlation between crowns and trophy gain: {correlation:.3f}")
print(f"  • Most common win type: {crown_analysis.loc[crown_analysis['battle_count'].idxmax(), 'crowns']}-crown win ({crown_analysis['pct_of_total'].max():.1f}% of battles)")

# Calculate trophy gain per crown
crown_analysis['trophy_per_crown'] = crown_analysis['avg_trophy_gain'] / crown_analysis['crowns']
print(f"  • Trophy gain per crown:")
for _, row in crown_analysis.iterrows():
    print(f"    - {row['crowns']} crown(s): {row['trophy_per_crown']:.2f} trophies per crown")

# Identify if 3-crown wins are significantly more rewarding
if len(crown_analysis) >= 3:
    three_crown_gain = crown_analysis[crown_analysis['crowns'] == 3]['avg_trophy_gain'].iloc[0]
    one_crown_gain = crown_analysis[crown_analysis['crowns'] == 1]['avg_trophy_gain'].iloc[0]
    bonus = three_crown_gain - (one_crown_gain * 3)
    print(f"  • 3-crown bonus: {bonus:.2f} extra trophies compared to 3x 1-crown wins")

## 5. Key Insights Summary

**Document your findings:**

### Insight 1: Arena Concentration
- **Finding**: A single arena (54000050) dominates the dataset, representing ~94% of all battles
- **Data**: Arena distribution shows extreme skew toward top-ladder play
- **Why it matters**: Insights primarily reflect high-skill meta; extrapolation to lower arenas requires caution. Game designers should consider balancing for different skill levels separately.

### Insight 2: Trophy Progression Walls
- **Finding**: Players cluster in specific trophy brackets, indicating progression bottlenecks
- **Data**: Trophy bracket analysis reveals concentration in 4000-5000 range
- **Why it matters**: Identifies where players struggle to progress, suggesting potential matchmaking or reward structure issues. Can inform game balance decisions.

### Insight 3: Crown-Trophy Correlation
- **Finding**: Higher crown wins correlate with higher trophy gains, but with diminishing returns
- **Data**: 3-crown wins provide bonus trophies beyond linear scaling
- **Why it matters**: Rewards aggressive playstyles; players may optimize for 3-crown strategies at certain trophy levels. Designers can adjust reward curves to encourage desired play patterns.

### Insight 4: Game Mode Risk/Reward Profiles
- **Finding**: Different game modes have distinct risk/reward characteristics
- **Data**: Trophy gain variance differs significantly by mode
- **Why it matters**: Players can choose modes matching their risk tolerance. Designers can balance modes to ensure all remain viable options.

### Visualizations to Create:
1. **Arena Battle Distribution Heatmap**: Show battle volume vs trophy gain across arenas
2. **Trophy Progression Funnel**: Visualize player concentration at different trophy brackets
3. **Crown Win Type Distribution**: Pie chart showing 1-crown vs 2-crown vs 3-crown percentages
4. **Game Mode Comparison Dashboard**: Side-by-side metrics for each game mode
5. **Trophy Gain Distribution by Crown Count**: Box plots showing variance in trophy gains